In [1]:
# Import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json

# Change this certs path to wherever you're storing your certificates
certs_path = "/home/naing/certs/"

# Change these login details to your own
my_username = "mightytranks@gmail.com"
my_password = "OsGiwuevzRf4925Lfty!"
my_app_key = "X9EGwt5zbpg8o5xk" #live=XzTt9hf8DAsmOuUa

trading = betfairlightweight.APIClient(username=my_username,
                                       password=my_password,
                                       app_key=my_app_key,
                                       certs=certs_path)

trading.login()


<LoginResource>

In [2]:
gh_racing_id = 4339
# Filter for just horse racing
greyhound_racing_filter = filters.market_filter(
    event_type_ids=[gh_racing_id],
    market_countries=['GB'],
    market_type_codes=['FORECAST'],
    market_start_time={
        'to': (datetime.datetime.utcnow() + datetime.timedelta(hours=6)).strftime("%Y-%m-%dT%TZ")
    }
)
greyhound_racing_filter

{'eventTypeIds': [4339],
 'marketCountries': ['GB'],
 'marketStartTime': {'to': '2022-06-19T19:40:53Z'}}

In [3]:
#  This returns a list
gb_gh_events = trading.betting.list_events(
    filter=greyhound_racing_filter)

# Create a DataFrame with all the events by iterating over each event object
gb_gh_events_df = pd.DataFrame({
    'Event Name': [event_object.event.name for event_object in gb_gh_events],
    'Event ID': [event_object.event.id for event_object in gb_gh_events],
    'Event Venue': [event_object.event.venue for event_object in gb_gh_events],
    'Country Code': [event_object.event.country_code for event_object in gb_gh_events],
    'Time Zone': [event_object.event.time_zone for event_object in gb_gh_events],
    'Open Date': [event_object.event.open_date for event_object in gb_gh_events],
    'Market Count': [event_object.market_count for event_object in gb_gh_events]
})

gb_gh_events_df

,Event Name,Event ID,Event Venue,Country Code,Time Zone,Open Date,Market Count
0,Kinsley 19th Jun,31536081,Kinsley,GB,Europe/London,2022-06-19 13:04:00,18
1,Swindon (F/C) 19th Jun,31536112,Swindon,GB,Europe/London,2022-06-19 17:19:00,9
2,Kinsley (F/C) 19th Jun,31536083,Kinsley,GB,Europe/London,2022-06-19 13:04:00,9
3,Henlow (F/C) 19th Jun,31536131,Henlow,GB,Europe/London,2022-06-19 17:17:00,8
4,Towcester 19th Jun,31536087,Towcester,GB,Europe/London,2022-06-19 12:57:00,22
5,Central Park 19th Jun,31536118,Central Park,GB,Europe/London,2022-06-19 17:09:00,18
6,Swindon 19th Jun,31536104,Swindon,GB,Europe/London,2022-06-19 17:19:00,18
7,Trap challenge,27640092,None,GB,Europe/London,2022-02-25 19:46:00,2
8,Central Park (F/C) 19th Jun,31536125,Central Park,GB,Europe/London,2022-06-19 17:09:00,9
9,Towcester (F/C) 19th Jun,31536095,Towcester,GB,Europe/London,2022-06-19 12:57:00,11


In [5]:
#TODO: extract a list of IDs for the forecast markets
fc_venue_ids = []
fc_venue_names = []
for eventObj in gb_gh_events:
    if '(F/C)' in eventObj.event.name:
        fc_venue_ids.append(eventObj.event.id)
        fc_venue_names.append(eventObj.event.name)

fc_venue_ids, fc_venue_names


(['31512294',
  '31512262',
  '31512300',
  '31508237',
  '31512366',
  '31512240',
  '31512337',
  '31512243',
  '31512279',
  '31512251',
  '31512347',
  '31512286'],
 ['Hove (F/C) 8th Jun',
  'Crayford (F/C) 8th Jun',
  'Yarmouth (F/C) 8th Jun',
  'Monmore (F/C) 6th Jun',
  'Sunderland (F/C) 8th Jun',
  'Nottingham (F/C) 8th Jun',
  'Doncaster (F/C) 8th Jun',
  'Swindon (F/C) 8th Jun',
  'Newcastle (F/C) 8th Jun',
  'Harlow (F/C) 8th Jun',
  'Harlow (F/C) 8th Jun',
  'Monmore (F/C) 8th Jun'])

In [6]:
# pick the random venue
import random
random_pick = random.randint(1,len(fc_venue_ids))
# random_pick = 1
venueOfTheDay_name = fc_venue_names[random_pick]
venueOfTheDay_id = fc_venue_ids[random_pick]

venueOfTheDay_name, venueOfTheDay_id

# market_types_filter = filters.market_filter(event_ids=[venueOfTheDay_id])

# market_types = trading.betting.list_market_types(
#     filter=market_types_filter
# )

# market_types_venueOfTheDay = pd.DataFrame({
#     'Market Type': [market_type_object.market_type for market_type_object in market_types],
# })
# market_types_venueOfTheDay

('Yarmouth (F/C) 8th Jun', '31512300')

In [8]:
market_catalogue_filter = filters.market_filter(event_ids=[venueOfTheDay_id])

market_catalogues = trading.betting.list_market_catalogue(
    filter=market_catalogue_filter,
    max_results='100',
    sort='FIRST_TO_START'
)

market_types_venueOfTheDay_df = pd.DataFrame({
    'Market Name': [market_cat_object.market_name for market_cat_object in market_catalogues],
    'Market ID': [market_cat_object.market_id for market_cat_object in market_catalogues],
    'Market Start Time': [market_cat_object.market_start_time for market_cat_object in market_catalogues],
    'Total Matched': [market_cat_object.total_matched for market_cat_object in market_catalogues],
    
})
market_types_venueOfTheDay_df

,Market Name,Market ID,Market Start Time,Total Matched
0,Forecast,1.199996501,None,0.0
1,Forecast,1.199996502,None,0.0
2,Forecast,1.199996503,None,0.0
3,Forecast,1.199996504,None,0.0
4,Forecast,1.199996505,None,0.0
5,Forecast,1.199996506,None,0.0
6,Forecast,1.199996507,None,0.0
7,Forecast,1.199996508,None,0.0
8,Forecast,1.199996509,None,0.0
9,Forecast,1.199996510,None,0.0


In [9]:
#TODO: extract a list of IDs for the forecast markets
fc_race_ids = []
fc_race_names = []
for raceObj in market_catalogues:
    fc_race_ids.append(raceObj.market_id)

fc_race_ids

['1.199996501',
 '1.199996502',
 '1.199996503',
 '1.199996504',
 '1.199996505',
 '1.199996506',
 '1.199996507',
 '1.199996508',
 '1.199996509',
 '1.199996510',
 '1.199996511',
 '1.199996512']

In [10]:
# pick the random race id
random_pick = random.randint(1,len(fc_race_ids))
myRaceID = fc_race_ids[random_pick]
myRaceID

'1.199996511'

In [175]:
def process_runner_books(runner_books):
    '''
    This function processes the runner books and returns a DataFrame with the best back/lay prices + vol for each runner
    :param runner_books:
    :return:
    '''
    best_back_prices = [runner_book.ex.available_to_back[0].price
                        if runner_book.ex.available_to_back
                        else 1.01
                        for runner_book
                        in runner_books]
    best_back_sizes = [runner_book.ex.available_to_back[0].size
                       if runner_book.ex.available_to_back
                       else 1.01
                       for runner_book
                       in runner_books]

    best_lay_prices = [runner_book.ex.available_to_lay[0].price
                       if runner_book.ex.available_to_lay
                       else 1000.0
                       for runner_book
                       in runner_books]
    best_lay_sizes = [runner_book.ex.available_to_lay[0].size
                      if runner_book.ex.available_to_lay
                      else 1.01
                      for runner_book
                      in runner_books]
    
    selection_ids = [runner_book.selection_id for runner_book in runner_books]
    last_prices_traded = [runner_book.last_price_traded for runner_book in runner_books]
    total_matched = [runner_book.total_matched for runner_book in runner_books]
    statuses = [runner_book.status for runner_book in runner_books]
    scratching_datetimes = [runner_book.removal_date for runner_book in runner_books]
    adjustment_factors = [runner_book.adjustment_factor for runner_book in runner_books]

    df = pd.DataFrame({
        'Selection ID': selection_ids,
        'Best Back Price': best_back_prices,
        'Best Back Size': best_back_sizes,
        'Best Lay Price': best_lay_prices,
        'Best Lay Size': best_lay_sizes,
        'Last Price Traded': last_prices_traded,
        'Total Matched': total_matched,
        'Status': statuses,
        'Removal Date': scratching_datetimes,
        'Adjustment Factor': adjustment_factors
    })
    return df

In [ ]:
# Create a price filter. Get all traded and offer data
price_filter = filters.price_projection(
    price_data=['EX_BEST_OFFERS']
)

# Request market books
market_books = trading.betting.list_market_book(
    market_ids=[myRaceID],
    price_projection=price_filter
)

# Grab the first market book from the returned list as we only requested one market 
market_book = market_books[0]

runners_df = process_runner_books(market_book.runners)

runners_df

,Selection ID,Best Back Price,Best Back Size,Best Lay Price,Best Lay Size,Last Price Traded,Total Matched,Status,Removal Date,Adjustment Factor
0,39483384,1.03,173.9,1000.0,1.01,None,0.0,ACTIVE,None,None
1,45404505,1.03,173.9,1000.0,1.01,None,0.0,ACTIVE,None,None
2,36800833,1.03,173.9,1000.0,1.01,None,0.0,ACTIVE,None,None
3,42466513,1.03,173.9,1000.0,1.01,None,0.0,ACTIVE,None,None
4,39983458,1.03,173.9,1000.0,1.01,None,0.0,ACTIVE,None,None
5,40839531,1.03,173.9,1000.0,1.01,None,0.0,ACTIVE,None,None


In [ ]:
fav_selection_id = runners_df.loc[runners_df['Best Lay Price'].idxmin(), 'Selection ID']
fav_price = runners_df.loc[runners_df['Best Lay Price'].idxmin(), 'Best Lay Price']

In [ ]:
limit_order_filter = filters.limit_order(
    size=5,
    price=3,
    persistence_type='LAPSE'
)

instructions_filter = filters.place_instruction(
    selection_id = str(fav_selection_id),
    order_type = "LIMIT",
    side="LAY",
    limit_order=limit_order_filter
)

instructions_filter

{'orderType': 'LIMIT',
 'selectionId': '39483384',
 'side': 'LAY',
 'limitOrder': {'price': 3, 'persistenceType': 'LAPSE', 'size': 5}}

In [ ]:
# order = trading.betting.place_orders(
#     market_id = myRaceID,
#     customer_strategy_ref='Naing_maker',
#     instructions=[instructions_filter]
# )

In [ ]:
# order.__dict__

In [ ]:
# trading.betting.list_current_orders(customer_strategy_refs=['Naing_maker']).__dict__

In [ ]:
# cancelled_order = trading.betting.cancel_orders(market_id=myRaceID)

In [ ]:
# # listClearedOrders
# cleared_orders = trading.betting.list_cleared_orders(bet_status="CANCELLED",
#                                                     market_ids=[myRaceID])
# # Create a DataFrame from the orders
# pd.DataFrame(cleared_orders._data['cancelledOrders'])
# cleared_orders